# HW 3 - Neural POS Tagger

In this exercise, you are going to build a set of deep learning models on part-of-speech (POS) tagging using Tensorflow 2. Tensorflow is a deep learning framwork developed by Google to provide an easier way to use standard layers and networks.

To complete this exercise, you will need to build deep learning models for POS tagging in Thai using NECTEC's ORCHID corpus. You will build one model for each of the following type:

- Neural POS Tagging with Word Embedding using Fixed / non-Fixed Pretrained weights
- Neural POS Tagging with Viterbi / Marginal CRF

Pretrained word embeddding are already given for you to use (albeit, a very bad one).

We also provide the code for data cleaning, preprocessing and some starter code for tensorflow 2 in this notebook but feel free to modify those parts to suit your needs. Feel free to use additional libraries (e.g. scikit-learn) as long as you have a model for each type mentioned above.

### Don't forget to change hardware accelrator to GPU in runtime on Google Colab ###

## 1. Setup and Preprocessing

We use POS data from [ORCHID corpus](https://www.nectec.or.th/corpus/index.php?league=pm), which is a POS corpus for Thai language.
A method used to read the corpus into a list of sentences with (word, POS) pairs have been implemented already. The example usage has shown below.
We also create a word vector for unknown word by random.

In [ ]:
%tensorflow_version 2.x

In [ ]:

!gdown --id 1tsfqDG8-HL4nkq0pq0HGifND_qDSHq-f
!unzip resources.zip 

Downloading...
From: https://drive.google.com/uc?id=1tsfqDG8-HL4nkq0pq0HGifND_qDSHq-f
To: /content/resources.zip
153MB [00:02, 70.0MB/s]
Archive:  resources.zip
   creating: resources/
   creating: resources/embeddings/
  inflating: resources/embeddings/emb_reader.py  
   creating: __MACOSX/
   creating: __MACOSX/resources/
   creating: __MACOSX/resources/embeddings/
  inflating: __MACOSX/resources/embeddings/._emb_reader.py  
 extracting: resources/embeddings/__init__.py  
  inflating: __MACOSX/resources/embeddings/.___init__.py  
   creating: resources/embeddings/__pycache__/
  inflating: resources/embeddings/__pycache__/__init__.cpython-36.pyc  
   creating: __MACOSX/resources/embeddings/__pycache__/
  inflating: __MACOSX/resources/embeddings/__pycache__/.___init__.cpython-36.pyc  
  inflating: resources/embeddings/__pycache__/emb_reader.cpython-36.pyc  
  inflating: __MACOSX/resources/embeddings/__pycache__/._emb_reader.cpython-36.pyc  
  inflating: __MACOSX/resources/embeddings/._

In [ ]:
!pip install python-crfsuite
!pip install tensorflow-addons
!pip install tf2crf

     |████████████████████████████████| 747kB 5.7MB/s 
     |████████████████████████████████| 706kB 4.2MB/s 


In [ ]:
%tensorflow_version 2.x

In [ ]:
from resources.data.orchid_corpus import get_sentences

import numpy as np
import numpy.random
import tensorflow as tf
np.random.seed(42)

In [ ]:
yunk_emb =np.random.randn(32)
train_data = get_sentences('train')
test_data = get_sentences('test')
print(train_data[0])

[('การ', 'FIXN'), ('ประชุม', 'VACT'), ('ทาง', 'NCMN'), ('วิชาการ', 'NCMN'), ('<space>', 'PUNC'), ('ครั้ง', 'CFQC'), ('ที่ 1', 'DONM')]


Next, we load pretrained weight embedding using pickle. The pretrained weight is a dictionary which map a word to its embedding.

In [ ]:
import pickle
fp = open('resources/basic_ff_embedding.pt', 'rb')
embeddings = pickle.load(fp)
fp.close()

The given code below generates an indexed dataset(each word is represented by a number) for training and testing data. The index 0 is reserved for padding to help with variable length sequence. (Additionally, You can read more about padding here [https://machinelearningmastery.com/data-preparation-variable-length-input-sequences-sequence-prediction/])

## 2. Prepare Data

In [ ]:
word_to_idx ={}
idx_to_word ={}
label_to_idx = {}
for sentence in train_data:
    for word,pos in sentence:
        if word not in word_to_idx:
            word_to_idx[word] = len(word_to_idx)+1
            idx_to_word[word_to_idx[word]] = word
        if pos not in label_to_idx:
            label_to_idx[pos] = len(label_to_idx)+1
word_to_idx['UNK'] = len(word_to_idx)

n_classes = len(label_to_idx.keys())+1
print(n_classes)

48


This section is tweaked a little from the demo, word2features will return word index instead of features, and sent2labels will return a sequence of word indices in the sentence.

In [ ]:
def word2features(sent, i, emb):
    word = sent[i][0]
    if word in word_to_idx :
        return word_to_idx[word]
    else :
        return word_to_idx['UNK']

def sent2features(sent, emb_dict):
    return np.asarray([word2features(sent, i, emb_dict) for i in range(len(sent))])

def sent2labels(sent):
    return numpy.asarray([label_to_idx[label] for (word, label) in sent],dtype='int32')

def sent2tokens(sent):
    return [word for (word, label) in sent]

In [ ]:
sent2features(train_data[100], embeddings)

array([ 29, 327,   5, 328])

Next we create train and test dataset, then we use tensorflow 2 to post-pad the sequence to max sequence with 0. Our labels are changed to a one-hot vector.

In [ ]:
%%time
x_train = np.asarray([sent2features(sent, embeddings) for sent in train_data])
y_train = [sent2labels(sent) for sent in train_data]
x_test = [sent2features(sent, embeddings) for sent in test_data]
y_test = [sent2labels(sent) for sent in test_data]

CPU times: user 308 ms, sys: 0 ns, total: 308 ms
Wall time: 314 ms


/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [ ]:
x_train=tf.keras.preprocessing.sequence.pad_sequences(x_train, maxlen=None, dtype='int32', padding='post', truncating='pre', value=0.)
y_train=tf.keras.preprocessing.sequence.pad_sequences(y_train, maxlen=None, dtype='int32', padding='post', truncating='pre', value=0.)
x_test=tf.keras.preprocessing.sequence.pad_sequences(x_test, maxlen=102, dtype='int32', padding='post', truncating='pre', value=0.)
y_temp =[]
for i in range(len(y_train)):
    y_temp.append(np.eye(n_classes)[y_train[i]][np.newaxis,:])
y_train = np.asarray(y_temp).reshape(-1,102,n_classes)
del(y_temp)

In [ ]:
print(x_train[100],x_train.shape)
print(y_train[100][3],y_train.shape)

[ 29 327   5 328   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0] (18500, 102)
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] (18500, 102, 48)


## 3. Evaluate

Our output from tf keras is a distribution of problabilities on all possible label. outputToLabel will return an indices of maximum problability from output sequence.

evaluation_report is the same as in the demo

In [ ]:
def outputToLabel(yt,seq_len):
    out = []
    for i in range(0,len(yt)):
        if(i==seq_len):
            break
        out.append(np.argmax(yt[i]))
    return out

In [ ]:
import pandas as pd
from IPython.display import display

def evaluation_report(y_true, y_pred):
    # retrieve all tags in y_true
    tag_set = set()
    for sent in y_true:
        for tag in sent:
            tag_set.add(tag)
    for sent in y_pred:
        for tag in sent:
            tag_set.add(tag)
    tag_list = sorted(list(tag_set))
    
    # count correct points
    tag_info = dict()
    for tag in tag_list:
        tag_info[tag] = {'correct_tagged': 0, 'y_true': 0, 'y_pred': 0}

    all_correct = 0
    all_count = sum([len(sent) for sent in y_true])
    for sent_true, sent_pred in zip(y_true, y_pred):
        for tag_true, tag_pred in zip(sent_true, sent_pred):
            if tag_true == tag_pred:
                tag_info[tag_true]['correct_tagged'] += 1
                all_correct += 1
            tag_info[tag_true]['y_true'] += 1
            tag_info[tag_pred]['y_pred'] += 1
    accuracy = (all_correct / all_count) * 100
            
    # summarize and make evaluation result
    eval_list = list()
    for tag in tag_list:
        eval_result = dict()
        eval_result['tag'] = tag
        eval_result['correct_count'] = tag_info[tag]['correct_tagged']
        precision = (tag_info[tag]['correct_tagged']/tag_info[tag]['y_pred'])*100 if tag_info[tag]['y_pred'] else '-'
        recall = (tag_info[tag]['correct_tagged']/tag_info[tag]['y_true'])*100 if (tag_info[tag]['y_true'] > 0) else 0
        eval_result['precision'] = precision
        eval_result['recall'] = recall
        eval_result['f_score'] = (2*precision*recall)/(precision+recall) if (type(precision) is float and recall > 0) else '-'
        
        eval_list.append(eval_result)

    eval_list.append({'tag': 'accuracy=%.2f' % accuracy, 'correct_count': '', 'precision': '', 'recall': '', 'f_score': ''})
    
    df = pd.DataFrame.from_dict(eval_list)
    df = df[['tag', 'precision', 'recall', 'f_score', 'correct_count']]
    display(df)

## 4. Train a model

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Reshape, Activation, Input, Dense,GRU,Reshape,TimeDistributed,Bidirectional,Dropout,Masking
from tensorflow.keras.optimizers import Adam

The model is this section is separated to two groups

- Neural POS Tagger (4.1)
- Neural CRF POS Tagger (4.2)

## 4.1.1 Neural POS Tagger  (Example)

We create a simple Neural POS Tagger as an example for you. This model dosen't use any pretrained word embbeding so it need to use Embedding layer to train the word embedding from scratch.

Instead of using tensorflow.keras.models.Sequential, we use tensorflow.keras.models.Model. The latter is better as it can have multiple input/output, of which Sequential model could not. Due to this reason, the Model class is widely used for building a complex deep learning model.

In [ ]:
inputs = Input(shape=(102,), dtype='int32')
output = (Embedding(len(word_to_idx),32,input_length=102,mask_zero=True))(inputs)
output = Bidirectional(GRU(32, return_sequences=True))(output)
output = Dropout(0.2)(output)
output = TimeDistributed(Dense(n_classes,activation='softmax'))(output)
model = Model(inputs, output)
model.compile(optimizer=Adam(lr=0.001),  loss='categorical_crossentropy', metrics=['categorical_accuracy'])

model.summary()
model.fit(x_train,y=y_train, batch_size=64,epochs=10,verbose=1)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 102)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 102, 32)           480608    
_________________________________________________________________
bidirectional (Bidirectional (None, 102, 64)           12672     
_________________________________________________________________
dropout (Dropout)            (None, 102, 64)           0         
_________________________________________________________________
time_distributed (TimeDistri (None, 102, 48)           3120      
Total params: 496,400
Trainable params: 496,400
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
290/290 [==============================] - 46s 135ms/step - loss: 0.3918 - categorical_accuracy: 0.3598
Epoch 

In [ ]:
%%time
model.fit(x_train,y_train,batch_size=64,epochs=10,verbose=1)

Epoch 1/10
290/290 [==============================] - 34s 119ms/step - loss: 0.0176 - categorical_accuracy: 0.9624
Epoch 2/10
290/290 [==============================] - 34s 118ms/step - loss: 0.0169 - categorical_accuracy: 0.9641
Epoch 3/10
290/290 [==============================] - 34s 118ms/step - loss: 0.0164 - categorical_accuracy: 0.9649
Epoch 4/10
290/290 [==============================] - 35s 120ms/step - loss: 0.0157 - categorical_accuracy: 0.9663
Epoch 5/10
290/290 [==============================] - 34s 118ms/step - loss: 0.0152 - categorical_accuracy: 0.9673
Epoch 6/10
290/290 [==============================] - 34s 119ms/step - loss: 0.0147 - categorical_accuracy: 0.9687
Epoch 7/10
290/290 [==============================] - 34s 119ms/step - loss: 0.0143 - categorical_accuracy: 0.9691
Epoch 8/10
290/290 [==============================] - 34s 118ms/step - loss: 0.0138 - categorical_accuracy: 0.9702
Epoch 9/10
290/290 [==============================] - 35s 119ms/step - loss: 0.0

In [ ]:
%%time
#model.save_weights('/data/my_pos_no_crf.h5')
#model.load_weights('/data/my_pos_no_crf.h5')
y_pred=model.predict(x_test)
ypred = [outputToLabel(y_pred[i],len(y_test[i])) for i in range(len(y_pred))]
evaluation_report(y_test, ypred)

,tag,precision,recall,f_score,correct_count
0,1,99.8099,99.7558,99.7828,3676
1,2,94.1769,94.1198,94.1483,7763
2,3,91.23,95.7783,93.4489,16176
3,4,99.9845,99.5357,99.7596,12862
4,5,87.8378,97.0149,92.1986,65
5,6,99.359,89.0805,93.9394,465
6,7,96.9334,97.3064,97.1195,2023
7,8,80,52.0482,63.0657,216
8,9,65.2632,67.3913,66.3102,248
9,10,61.1212,40.2861,48.5632,338


CPU times: user 7.32 s, sys: 247 ms, total: 7.57 s
Wall time: 5.36 s


## 4.2 CRF Viterbi

Your next task is to incorporate Conditional random fields (CRF) to your model.

To use the CRF layer, you need to use an extension repository for tensorflow library, call tf2crf. If you want to see the detailed implementation, you should read the official tensorflow extention of CRF (https://www.tensorflow.org/addons/api_docs/python/tfa/text).

tf2crf link :  https://github.com/xuxingya/tf2crf

For inference, you should look at crf.py at the method call and view the input/output argmunets. 
Link : https://github.com/xuxingya/tf2crf/blob/master/tf2crf/crf.py



### 4.2.1 CRF without pretrained weight
### #TODO 1
Incoperate CRF layer to your model in 4.1. CRF is quite complex compare to previous example model, so you should train it with more epoch, so it can converge.

To finish this excercise you must train the model and show the evaluation report with this model as shown in the example.

Do not forget to save this model weight.

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Reshape, Activation, Input, Dense,GRU,Reshape,TimeDistributed,Bidirectional,Dropout,Masking
from tensorflow.keras.optimizers import Adam
from tf2crf import CRF, ModelWithCRFLoss
# INSERT YOUR CODE HERE
inputs = Input(shape=(102,), dtype='int32')
output = (Embedding(len(word_to_idx),32,input_length=102,mask_zero=True))(inputs)
output = Bidirectional(GRU(32, return_sequences=True))(output)
output = Dropout(0.2)(output)
#output = Dense(n_classes, activation=None)(output)
output = TimeDistributed(Dense(n_classes,activation=None))(output)
crf = CRF(dtype='float32')
output = crf(output)
base_model = Model(inputs, output)
model = ModelWithCRFLoss(base_model)
#model.compile(optimizer=Adam(lr=0.001),  loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.compile(optimizer='adam')

y_crf_train = [[np.argmax(word) for word in sentence ] for sentence in y_train]
y_crf_train = tf.keras.preprocessing.sequence.pad_sequences(y_crf_train, maxlen=None, dtype='int32', padding='post', truncating='pre', value=0.)

#model.summary()
model.fit(x_train,y=y_crf_train, batch_size=64,epochs=10,verbose=1)


Epoch 1/10
290/290 [==============================] - 92s 287ms/step - crf_loss: 26.6608 - accuracy: 0.5541
Epoch 2/10
290/290 [==============================] - 84s 291ms/step - crf_loss: 5.8926 - accuracy: 0.9063
Epoch 3/10
290/290 [==============================] - 84s 290ms/step - crf_loss: 3.5733 - accuracy: 0.9368
Epoch 4/10
290/290 [==============================] - 84s 289ms/step - crf_loss: 2.8150 - accuracy: 0.9470
Epoch 5/10
290/290 [==============================] - 84s 291ms/step - crf_loss: 2.4819 - accuracy: 0.9518
Epoch 6/10
290/290 [==============================] - 84s 291ms/step - crf_loss: 2.2779 - accuracy: 0.9545
Epoch 7/10
290/290 [==============================] - 84s 291ms/step - crf_loss: 2.1243 - accuracy: 0.9568
Epoch 8/10
290/290 [==============================] - 84s 291ms/step - crf_loss: 2.0133 - accuracy: 0.9585
Epoch 9/10
290/290 [==============================] - 85s 291ms/step - crf_loss: 1.9214 - accuracy: 0.9602
Epoch 10/10
290/290 [===============

In [ ]:
#model.save_weights('/data/my_pos_no_crf.h5')
#model.load_weights('/data/my_pos_no_crf.h5')
y_pred=model.predict(x_test)

evaluation_report(y_test, y_pred[0])

,tag,precision,recall,f_score,correct_count
0,0,-,0,-,0
1,1,99.864,99.6201,99.7419,3671
2,2,95.3181,93.55,94.4257,7716
3,3,91.1172,95.2336,93.1299,16084
4,4,99.8759,99.6363,99.7559,12875
5,5,95.6522,98.5075,97.0588,66
6,6,99.7802,86.9732,92.9376,454
7,7,98.0985,96.7773,97.4334,2012
8,8,70.1863,54.4578,61.3297,226
9,9,76.1246,59.7826,66.9711,220



### 4.2.2 CRF with pretrained weight

### #TODO 2

We would like you create a neural CRF POS tagger model  with the pretrained word embedding as an input and the word embedding is trainable (not fixed). To finish this excercise you must train the model and show the evaluation report with this model as shown in the example.

Please note that the given pretrained word embedding only have weights for the vocabuary in BEST corpus.

Optionally, you can use your own pretrained word embedding.

#### Hint: You can get the embedding from get_embeddings function from embeddings/emb_reader.py . 

(You may want to read about Tensorflow Masking layer and Trainable parameter)

In [ ]:
# INSERT YOUR CODE HERE
from resources.embeddings.emb_reader import get_embeddings
embeddings_index = get_embeddings()
embedding_matrix = np.zeros((len(word_to_idx) , 64))

for word, i in word_to_idx.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector


In [ ]:

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Reshape, Activation, Input, Dense,GRU,Reshape,TimeDistributed,Bidirectional,Dropout,Masking
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import Constant
from tf2crf import CRF, ModelWithCRFLoss

inputs = Input(shape=(102,), dtype='int32')
#output = (Embedding(len(word_to_idx),64,weights=[embedding_matrix],input_length=102,trainable=False,mask_zero=True))(inputs)
output = (Embedding(len(word_to_idx),64,embeddings_initializer=Constant(embedding_matrix),input_length=102,trainable=True,mask_zero=True))(inputs)
output = Bidirectional(GRU(64, return_sequences=True))(output)
output = Dropout(0.2)(output)
output = TimeDistributed(Dense(n_classes,activation=None))(output)
crf = CRF(dtype='float32')
output = crf(output)
base_model = Model(inputs, output)
model = ModelWithCRFLoss(base_model)
model.compile(optimizer='adam')

y_crf_train = [[np.argmax(word) for word in sentence ] for sentence in y_train]
y_crf_train = tf.keras.preprocessing.sequence.pad_sequences(y_crf_train, maxlen=None, dtype='int32', padding='post', truncating='pre', value=0.)

#model.summary()
model.fit(x_train,y=y_crf_train, batch_size=64,epochs=10,verbose=1)



Epoch 1/10
290/290 [==============================] - 113s 356ms/step - crf_loss: 18.3623 - accuracy: 0.6909
Epoch 2/10
290/290 [==============================] - 104s 360ms/step - crf_loss: 3.9581 - accuracy: 0.9291
Epoch 3/10
290/290 [==============================] - 104s 357ms/step - crf_loss: 2.7292 - accuracy: 0.9481
Epoch 4/10
290/290 [==============================] - 104s 360ms/step - crf_loss: 2.2935 - accuracy: 0.9548
Epoch 5/10
290/290 [==============================] - 105s 362ms/step - crf_loss: 2.0575 - accuracy: 0.9580
Epoch 6/10
290/290 [==============================] - 104s 359ms/step - crf_loss: 1.9137 - accuracy: 0.9602
Epoch 7/10
290/290 [==============================] - 104s 358ms/step - crf_loss: 1.7895 - accuracy: 0.9622
Epoch 8/10
290/290 [==============================] - 104s 357ms/step - crf_loss: 1.6840 - accuracy: 0.9645
Epoch 9/10
290/290 [==============================] - 104s 359ms/step - crf_loss: 1.5892 - accuracy: 0.9659
Epoch 10/10
290/290 [======

In [ ]:
y_pred=model.predict(x_test)
evaluation_report(y_test, y_pred[0])

,tag,precision,recall,f_score,correct_count
0,0,-,0,-,0
1,1,99.7828,99.7286,99.7557,3675
2,2,94.2497,95.1867,94.7159,7851
3,3,91.287,96.3408,93.7459,16271
4,4,99.9844,99.3886,99.6856,12843
5,5,82.5,98.5075,89.7959,66
6,6,99.5807,90.9962,95.0951,475
7,7,98.1086,97.3064,97.7059,2023
8,8,67.4359,63.3735,65.3416,263
9,9,58.8235,62.5,60.6061,230


### #TODO 3
Compare the result between all neural tagger models in 4.1 and 4.2.x and provide a convincing reason and example for the result of these models (which model perform better, why?)

(If you use your own weight please state so in the answer)

In [ ]:
print("จาก model ทั้งสามที่ได้ทำการ train และ predict ผล ได้ข้อสรุปว่า model ทั้งสามได้ accuracy ที่ไม่แตกต่างกันอย่างมีนัยสำคัญ")
print("โดย model ที่ดีที่สุดจะเป็น model ที่ได้ใส่ pre-train weight ลงไป อันเนื่องมาจาก pre-train weight ดังกล่าวนั้นได้รับการ train เพื่อให้ได้ค่าที่ถูกต้องมาก่อนแล้ว")
print("จึ่งทำให้ model ดังกล่าวได้ accuracy ที่ดีที่สุด(แต่กระนั้นก็ยังไม่ทำให้เกิดความแตกต่างแบบมีนัยสำคัญต่อ model อื่น)")


จาก model ทั้งสามที่ได้ทำการ train และ predict ผล ได้ข้อสรุปว่า model ทั้งสามได้ accuracy ที่ไม่แตกต่างกันอย่างมีนัยสำคัญ
โดย model ที่ดีที่สุดจะเป็น model ที่ได้ใส่ pre-train weight ลงไป อันเนื่องมาจาก pre-train weight ดังกล่าวนั้นได้รับการ train เพื่อให้ได้ค่าที่ถูกต้องมาก่อนแล้ว
จึ่งทำให้ model ดังกล่าวได้ accuracy ที่ดีที่สุด(แต่กระนั้นก็ยังไม่ทำให้เกิดความแตกต่างแบบมีนัยสำคัญต่อ model อื่น)


<b>Write your answer here :</b>

### #TODO 4

Upon inference, the model also returns its transition matrix, which is learned during training. Your task is to observe and report whether the returned matrix is sensible. You can provide some examples to support your argument.

#### Hint : The transition matrix must have the shape  of (num_class, num_class).

<b>Write your answer here :</b>

In [ ]:
import plotly.express as px
transition_matrix = np.array(crf.transitions)
fig = px.imshow(transition_matrix )
fig.update_xaxes(dtick=1)
fig.update_yaxes(dtick=1)
fig.update_layout(
    width=750,
    height=750
)
fig.show()
print(label_to_idx)



{'FIXN': 1, 'VACT': 2, 'NCMN': 3, 'PUNC': 4, 'CFQC': 5, 'DONM': 6, 'JCRG': 7, 'NCNM': 8, 'CNIT': 9, 'NPRP': 10, 'NTTL': 11, 'XVAM': 12, 'VSTA': 13, 'RPRE': 14, 'ADVN': 15, 'JSBR': 16, 'DDAC': 17, 'XVBM': 18, 'XVMM': 19, 'DIBQ': 20, 'PREL': 21, 'VATT': 22, 'XVAE': 23, 'DCNM': 24, 'CMTR': 25, 'FIXV': 26, 'PPRS': 27, 'XVBB': 28, 'DIAC': 29, 'PDMN': 30, 'DDAN': 31, 'CLTV': 32, 'ADVP': 33, 'NLBL': 34, 'ADVI': 35, 'CMTR@PUNC': 36, 'JCMP': 37, 'ADVS': 38, 'DDBQ': 39, 'NEG': 40, 'PNTR': 41, 'EITT': 42, 'DDAQ': 43, 'NONM': 44, 'EAFF': 45, 'DIAQ': 46, 'CVBL': 47}


In [ ]:
# INSERT YOUR CODE HERE IF NEEDED
print("transition matrix ที่แสดงออกมาจะเป็นการแสดงการเปลี่ยนจาก pos ของคำในปัจจุบันใน(แกน y) ไปยัง pos ของคำต่อไป(แกน x)")
print("โดย transition matrix ที่ได้ออกมานั้นมีค่าใกล้เคียงกับความเป็นจริง\nตัวอย่างเช่น")
print("x=22,y=26 \n   : FIXV -> VATT ซึ่งมีค่า",transition_matrix[26][22],"เป็นเหตุเป็นผลไปกับการที่ FIXVที่เป็นคำนำหน้าคำคุณศัพท์->VATTคำคุณศัพท์\n    อย่าง/FXIV->เร็ว/VATT")
print("x=10,y=11 \n   : NTTL -> NPRP ซึ่งมีค่า",transition_matrix[11][10],"เป็นเหตุเป็นผลไปกับการที่ NTTLที่เป็นการสื่อถึงคำนำหน้าชื่อของบุคคล->NPRPที่หมายถึงนามเฉพาะซึ่งรวมถึงชื่อบุคคล\n    นาย/NTTL->พลวัต/NPRP")

transition matrix ที่แสดงออกมาจะเป็นการแสดงการเปลี่ยนจาก pos ของคำในปัจจุบันใน(แกน y) ไปยัง pos ของคำต่อไป(แกน x)
โดย transition matrix ที่ได้ออกมานั้นมีค่าใกล้เคียงกับความเป็นจริง
ตัวอย่างเช่น
x=22,y=26 
   : FIXV -> VATT ซึ่งมีค่า 0.809802 เป็นเหตุเป็นผลไปกับการที่ FIXVที่เป็นคำนำหน้าคำคุณศัพท์->VATTคำคุณศัพท์
    อย่าง/FXIV->เร็ว/VATT
x=10,y=11 
   : NTTL -> NPRP ซึ่งมีค่า 0.65363085 เป็นเหตุเป็นผลไปกับการที่ NTTLที่เป็นการสื่อถึงคำนำหน้าชื่อของบุคคล->NPRPที่หมายถึงนามเฉพาะซึ่งรวมถึงชื่อบุคคล
    นาย/NTTL->พลวัต/NPRP
